In [90]:
from bs4 import BeautifulSoup
import os
import json

def readXml(filepath):
    return BeautifulSoup(open(filepath,encoding="utf8"))

def traceChildren(children):
    for child in children:
        print("~~~",child)
def traceObj(obj):
    print("traceObj:",obj)
    for key in obj:
        print(key)

def getNodeClass(node):
    try:
        return node["class"]
    except:
        return None;

def parseLeftUL(ul,childList):
    #print("parseLeftUL:",ul)
    #traceChildren(ul.children)
    
    for child in ul.children:
        aO=child.find("a")
        if aO==None:
            continue
        #print("aO:",aO)
        aObj={};
        aObj["href"]=aO["href"];
        aObj["title"]=aO.string
        childList.append(aObj)
    
def parseWikiLeft(leftPart):
    #traceChildren(leftPart.children)
    navList=[]
    for child in leftPart.children:
        #print("!!!",type(child),child.name)
        tChildClass=getNodeClass(child)
        if child.name=="ul":
            if tNavO:
                parseLeftUL(child,tNavO["child"])
            
            continue
            
        if tChildClass!=None:
            if "sidehead" in tChildClass:
                #print(child)
                if child.a:
                    break
                tNavO={};
                tNavO["title"]=child.string
                if tNavO["title"]!=None:
                    tNavO["title"]=tNavO["title"].strip()
                tNavO["child"]=[]
                navList.append(tNavO)
        #traceObj(child)
    #print(navList)
    return navList

def parseWikiText(textPart):
    navList=[]
    tNavO=None
    for child in textPart.children:
        #print("!!!",type(child),child.name)
        tChildClass=getNodeClass(child)
        if child.name=="ul":
            if tNavO:
                parseLeftUL(child,tNavO["child"])
            
            continue
            
        if tChildClass!=None:
            if "vspace" in tChildClass:
                #print(child)
                tNavO={};
                tNavO["title"]=child.string
                if tNavO["title"]!=None:
                    tNavO["title"]=tNavO["title"].strip()
                tNavO["child"]=[]
                navList.append(tNavO)
        #traceObj(child)
    #print("wikitext:",navList)
    return navList

def parseWikiTitle(titlePart):
    group=titlePart.find(None,"pagegroup")
    print("group:",group.a.string)
    title=titlePart.find(None,"pagetitle")
    print("title:",title.string.strip())
    infoO={};
    infoO["group"]=group.a.string.strip()
    infoO["title"]=title.string.strip()
    return infoO
    
def getStructDataFromHtmlO(xmlO):
    rstO={}
    leftPart=xmlO.find(id="wikileft")
    
    
    rstO["Nav"]=parseWikiLeft(leftPart)

    wikiTitle=xmlO.find(id="wikititle")
    #print(wikiTitle)
    rstO["Info"]=parseWikiTitle(wikiTitle)

    wikiText=xmlO.find(id="wikitext")
    #print(wikiText)
    rstO["Content"]=parseWikiText(wikiText)
    
    return rstO
 
def workHtmPage(patePath,fileName):
    xmlO=readXml(patePath)
    dataO=getStructDataFromHtmlO(xmlO)
    fw=open("website/"+fileName+".json","w",encoding="utf-8")
    json.dump(dataO,fw,ensure_ascii=False,indent=4)

def getDirFiles(dirPath):
    files=os.listdir(dirPath)
    print(files)
    for tfile in files:
        tfile=dirPath+"/"+tfile
        #print("isdir:",os.path.isdir(tfile))
        if os.path.isdir(tfile):
            continue
        print(tfile)
        bname=os.path.basename(os.path.realpath(tfile))
        print(bname)
        names=bname.split(".")
        names.pop()
        names=names[0].split(" _ ")
        names.pop(0)
        print(names)
        print("_".join(names))
        workHtmPage(tfile,"_".join(names))
#workHtmPage("wiki/OrzWiki _ 学习 _ 量化交易.htm")

getDirFiles("wiki")


['OrzWiki _ Main _ HomePage.htm', 'OrzWiki _ Main _ HomePage_files', 'OrzWiki _ Other _ Ideas.htm', 'OrzWiki _ Other _ Ideas_files', 'OrzWiki _ SNS _ 推荐引擎.htm', 'OrzWiki _ SNS _ 推荐引擎_files', 'OrzWiki _ SNS _ 网站相关.htm', 'OrzWiki _ SNS _ 网站相关_files', 'OrzWiki _ 学习 _ AR.htm', 'OrzWiki _ 学习 _ AR_files', 'OrzWiki _ 学习 _ As3.htm', 'OrzWiki _ 学习 _ As3_files', 'OrzWiki _ 学习 _ IDE.htm', 'OrzWiki _ 学习 _ IDE_files', 'OrzWiki _ 学习 _ Js.html', 'OrzWiki _ 学习 _ Js_files', 'OrzWiki _ 学习 _ Python.htm', 'OrzWiki _ 学习 _ Python_files', 'OrzWiki _ 学习 _ TodoList.htm', 'OrzWiki _ 学习 _ TodoList_files', 'OrzWiki _ 学习 _ 产品Amp运营.htm', 'OrzWiki _ 学习 _ 产品Amp运营_files', 'OrzWiki _ 学习 _ 公开课forDear.htm', 'OrzWiki _ 学习 _ 公开课forDear_files', 'OrzWiki _ 学习 _ 动画相关.htm', 'OrzWiki _ 学习 _ 动画相关_files', 'OrzWiki _ 学习 _ 安全相关.htm', 'OrzWiki _ 学习 _ 安全相关_files', 'OrzWiki _ 学习 _ 开放平台们.htm', 'OrzWiki _ 学习 _ 开放平台们_files', 'OrzWiki _ 学习 _ 数据分析.htm', 'OrzWiki _ 学习 _ 数据分析_files', 'OrzWiki _ 学习 _ 算法.htm', 'OrzWiki _ 学习 _ 算法_files', 'OrzWi

D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


wiki/OrzWiki _ SNS _ 推荐引擎.htm
OrzWiki _ SNS _ 推荐引擎.htm
['SNS', '推荐引擎']
SNS_推荐引擎
group: SNS
title: 推荐引擎
wiki/OrzWiki _ SNS _ 网站相关.htm
OrzWiki _ SNS _ 网站相关.htm
['SNS', '网站相关']
SNS_网站相关
group: SNS
title: 网站相关
wiki/OrzWiki _ 学习 _ AR.htm
OrzWiki _ 学习 _ AR.htm
['学习', 'AR']
学习_AR
group: 学习
title: AR
wiki/OrzWiki _ 学习 _ As3.htm
OrzWiki _ 学习 _ As3.htm
['学习', 'As3']
学习_As3
group: 学习
title: As3
wiki/OrzWiki _ 学习 _ IDE.htm
OrzWiki _ 学习 _ IDE.htm
['学习', 'IDE']
学习_IDE
group: 学习
title: IDE
wiki/OrzWiki _ 学习 _ Js.html
OrzWiki _ 学习 _ Js.html
['学习', 'Js']
学习_Js
group: 学习
title: Js
wiki/OrzWiki _ 学习 _ Python.htm
OrzWiki _ 学习 _ Python.htm
['学习', 'Python']
学习_Python
group: 学习
title: Python
wiki/OrzWiki _ 学习 _ TodoList.htm
OrzWiki _ 学习 _ TodoList.htm
['学习', 'TodoList']
学习_TodoList
group: 学习
title: TodoList
wiki/OrzWiki _ 学习 _ 产品Amp运营.htm
OrzWiki _ 学习 _ 产品Amp运营.htm
['学习', '产品Amp运营']
学习_产品Amp运营
group: 学习
title: 产品Amp运营
wiki/OrzWiki _ 学习 _ 公开课forDear.htm
OrzWiki _ 学习 _ 公开课forDear.htm
['学习', '公开课forDear']
学习_公开